In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import Symbol, Poly, diff, lambdify, Piecewise

%matplotlib inline

In [3]:
from 

ValueError: 
Can't calculate derivative wrt 0.234000000000000.

In [ ]:
import sympy
sympy.__version__

In [ ]:
class OdeHalfAnalyticalSolve:
    """ Only for temperature problem
        du/dt = a^2 d^2u/dt^2 """
    
    def __init__(self, k, U_left, U_right, B_0, t0, x0, step_t, step_x, T, L, degree=1):
        self.k = k
        self.U_left = U_left
        self.U_right = U_right
        self.B_0 = B_0
        self.t_part = np.arange(t0, T+step_t, step_t)
        self.x_part = np.arange(x0, L+step_x, step_x)
        self.T = T
        self.L = L
        self.degree = degree
        
    def get_polynomial_coeffs(self):
        pass
    
    def get_prime(self, f):
        P = f(Symbol('x'))
        prime = P.diff(Symbol('x'))
        return lambdify(Symbol('x'), prime)
    
    def solve_system(self):
        dim = 4*(self.x_part.shape[0]-1)
        A = np.zeros((dim, dim))
        B = np.zeros(dim)
        
        A[0][0] = 1
        A[-1][-2] = 1
        
        B[0] = self.U_left
        B[-1] = self.U_right
        
        A_conn = lambda x, b: np.array([[-np.cosh(x/b), b*np.sinh(x/b)/self.k, 1, 0],
                                         [self.k*np.sinh(x/b)/b, -np.cosh(x/b), 0, 1]])
        
        B_conn = lambda x: np.array([B_prev(x), -self.k*B_prev_prime(x)])
        
        A_conj = np.array([[1, 0, -1, 0], [0, 1, 0, -1]])
        
    def solve():
        for dt in self.t_part

In [ ]:
from sympy import Symbol, Poly, diff, lambdify, Piecewise
f = lambda x: x**3 + 2*x**2 - 5*x + 1
P = f(Symbol('x'))
prime = P.diff(Symbol('x'))
prime_lam = lambdify(Symbol('x'), prime)
prime_lam(1)

In [ ]:
prime

In [ ]:
f = lambda x: x if x <= 1 else 2 - x
P = f(Symbol('x'))